In [56]:
import re 
import string 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm
import json

from pandas.core.frame import DataFrame
from IPython.core.display import display, HTML


import re, nltk, spacy, gensim
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint


nlp = spacy.load("en_core_web_sm")

In [4]:
data= pd.read_csv("speeches.csv")
usage=data.sample(10000)

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8,10,11,12,15,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
usage.to_csv("sample.csv")

In [10]:
data = pd.read_csv('sample.csv')
data = data.sort_values('year')
data

,Unnamed: 0,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,speech_class,major_heading,minor_heading,oral_heading,year,hansard_membership_id,speakerid,person_id,speakername,url
6280,28237,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Labour,Caithness and Sutherland,578.0,1979-11-27,NaN,1096.0,Speech,Oral Answers to Questions — DEFENCE,Garvie Island,NaN,1979,8447.0,uk.org.publicwhip/member/2994,NaN,Mr. Maclennan,NaN
2137,28375,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Conservative,Faversham,922.0,1979-11-27,16:40:00,1136.0,Speech,Orders of the Day — TRANSPORT BILL,NaN,NaN,1979,9151.0,uk.org.publicwhip/member/18434,NaN,Mr. Roger Moate,NaN
9353,29105,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Labour,Oldham East,894.0,1979-11-29,NaN,1477.0,Speech,Oral Answers to Questions — ORAL ANSWERS TO QU...,PRIME MINISTER (ENGAGEMENTS),NaN,1979,7367.0,uk.org.publicwhip/member/16997,NaN,Mr. Lamond,NaN
6748,23328,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,NaN,1979-11-12,00:30:00,1080.0,Procedural,Clause 3,OTHER POWERS WITH RESPECT TO SOUTHERN RHODESIA,NaN,1979,NaN,NaN,NaN,Unknown,NaN
8012,29046,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,Conservative,North Angus and Mearns,727.0,1979-11-29,NaN,1470.0,Speech,Oral Answers to Questions — ORAL ANSWERS TO QU...,Pigs (Breeding Herd),NaN,1979,16624.0,uk.org.publicwhip/member/11624,NaN,Mr. Buchanan-Smith,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6095,2684453,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,NaN,2021-02-24,18:56:00,1024.0,Procedural,Telecommunications Infrastructure (Leasehold P...,NaN,NaN,2021,NaN,NaN,NaN,Unknown,NaN
6168,2693949,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Conservative,Banff and Buchan,4606.0,2021-04-28,NaN,361.0,Speech,Scotland,COP26,NaN,2021,NaN,NaN,uk.org.publicwhip/person/25664,David Duguid,NaN
6193,2683641,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Conservative,Uxbridge and South Ruislip,1423.0,2021-02-22,15:51:00,655.0,Speech,Covid-19: Road Map,NaN,NaN,2021,NaN,NaN,uk.org.publicwhip/person/10999,Boris Johnson,NaN
5839,2681899,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Conservative,Harlow,3985.0,2021-02-04,15:45:00,1186.0,Speech,Towns Fund,NaN,NaN,2021,NaN,NaN,uk.org.publicwhip/person/24784,Robert Halfon,NaN


In [11]:
def extract_noun(text):
        return [(chunk.text) for chunk in nlp(text).noun_chunks]
data['Noun'] = data['speech'].apply(extract_noun)

In [13]:
def extract_named_ents(text):
    return [(ent.text, ent.label_) for ent in nlp(text).ents]
data['Name_enitity'] = data['speech'].apply(extract_named_ents)

In [14]:
def subtree_matcher(doc):
    subjpass = 0

    for i,tok in enumerate(doc):
        
    # find dependency tag that contains the text "subjpass"    
       if tok.dep_.find("subjpass") == True:
            subjpass = 1

    x = ''
    y = ''
    z = ''

  # if subjpass == 1 then sentence is passive
    if subjpass == 1:
        for i,tok in enumerate(doc):
            if tok.dep_.find("subj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    y = doc[i-1].text + ' '+tok.text
                else:
                    y=tok.text
                                   
            if(tok.dep_.find('ROOT')==0):
                z=tok.text
                
            if tok.dep_.endswith("obj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    x = doc[i-1].text + ' '+tok.text  
                else:
                    x = tok.text
  
  # if subjpass == 0 then sentence is not passive
    else:
        for i,tok in enumerate(doc):
            if tok.dep_.endswith("subj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    x = doc[i-1].text + ' '+tok.text 
                else:
                    x = tok.text
                    
            if(tok.dep_.find('ROOT')==0):
                z=tok.text

            if tok.dep_.endswith("obj") == True:
                if(doc[i-1].dep_ in['amod','compound']):
                    y = doc[i-1].text + ' '+tok.text
                else:
                    y = tok.text

    return x,y,z

def get_relation(text):
    document = nlp(text)
    t = [sent.text for sent in document.sents]
    re =  []
    for i in t:
        i = nlp(i)
        re.append(subtree_matcher(i))
    return re

data['RE'] = data['speech'].apply(get_relation)   

In [16]:
def extract_verb(text):
    return [token for token in nlp(text)if token.pos_=='VERB']
data['Verb'] = data['speech'].apply(extract_verb)

In [17]:
def N_pre_N(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:

        # look for prepositions
        if token.pos_=='ADP':

            phrase = ''
            
            # if its head word is a noun
            if token.head.pos_=='NOUN':
                
                # append noun and preposition to phrase
                phrase += token.head.text
                phrase += ' '+token.text

                # check the nodes to the right of the preposition
                for right_tok in token.rights:
                    # append if it is a noun or proper noun
                    if (right_tok.pos_ in ['NOUN','PROPN']):
                        phrase += ' '+right_tok.text
                
                if len(phrase)>2:
                    sent.append(phrase)
                
    return sent
def get_r(text):
    document = nlp(text)
    t = [sent.text for sent in document.sents]
    re =  []
    for i in t:
        i = nlp(i)
        re.append(subtree_matcher(i))
    return re

data['RE2'] = data['speech'].apply(get_r) 

In [18]:
def get_pps(text):
    "Function to get PPs from a parsed document."
    pps = []
    doc = nlp(text)
    for token in doc:
        # Try this with other parts of speech for different subtrees.
        if token.pos_ == 'ADP':
            pp = ' '.join([tok.orth_ for tok in token.subtree])
            pps.append(pp)
    return pps
data['Pres'] = data['speech'].apply(get_pps) 

In [52]:
# topic matching
po = pd.read_csv('politics.csv')
del po['Unnamed: 0']
politics = po['term'].tolist()
# using pattern matching
def group(text):
    pattern = [r'\b(?i)'+ i +r'\b' for i in politics]

    
    flag = 0
    for pat in patterns:
        if re.search(pat, text) != None:
            flag = 1
            break
    return flag
 # flag 1 == politics
#flag 2 = environemnt and protection
#flag 3 = economy
#flag 4 = education
def topic(text):
    document = nlp(text)
    t = [sent.text for sent in document.sents]
    re = []
    for i in t:
        re.append(group(i))
    if 1 in re:
        return 1
    else:
        return None
    
data['topic'] = data['speech'].apply(topic)

<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)scheme\\b'
  if re.search(pat, text) != None:
<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)campaign\\b'
  if re.search(pat, text) != None:
<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)initiative\\b'
  if re.search(pat, text) != None:
<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)conference\\b'
  if re.search(pat, text) != None:
<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)agreement\\b'
  if re.search(pat, text) != None:
<ipython-input-52-38edb4384768>:12: DeprecationWarning: Flags not at the start of the expression '\\b(?i)alliance\\b'
  if re.search(pat, text) != None:


In [54]:
#specific expression
# rule to extract initiative name
def sent_subtree(text):
    
    # pattern match for schemes or initiatives
    pattern = [r'\b(?i)'+ i +r'\b' for i in politics]
    
    schemes = []
    doc = nlp(text)
    flag = 0
    # if no initiative present in sentence
    for pat in patterns:
        
        if re.search(pat, text) != None:
            flag = 1
            break
    
    if flag == 0:
        return schemes

    # iterating over sentence tokens
    for token in doc:

        for pat in patterns:
                
            # if we get a pattern match
            if re.search(pat, token.text) != None:

                word = ''
                # iterating over token subtree
                for node in token.subtree:
                    # only extract the proper nouns
                    if (node.pos_ == 'PROPN'):
                        word += node.text+' '

                if len(word)!=0:
                    schemes.append(word)

    return schemes      

def politicsPhase(text):
    document = nlp(text)
    t = [sent.text for sent in document.sents]
    re = []
    for i in t:
        re.append(sent_subtree(i))
    return re
    
data['politics_phase'] = data['speech'].apply(politicsPhase)

In [61]:


# subgroup data is too small, speicific classify with machine larning method, 
#1. td-idf, and LDA , using google colab to train and download the model.
#next work, visulasation and optimize
del data['Unnamed: 0']

In [ ]:
#LDA method frquency decide the topics
# remove stopwords
# Sklearn
#simple preprocessing
from spacy.lang.en.stop_words import STOP_WORDS
from textblob import TextBlob

#Implementation of stop words:
def remove_stop(text):
    filterd = []
    doc = nlp(text)
    for word in doc:
        if word.is_stop == False:
            filtered.append(word)
    return filtered
def lemma(text):
    text = nlp(text)
    sent = []
    for word in comment:
        sent.append(word.lemma_)    
    return " ".join(sent)
#keep only nouns
def nouns(text):
    blob = TextBlob(comment)
    x = blob.noun_phrases
    return " ".join(x)
def pos(text):
    text = nlp(text)
    nouns = []
    i=0
    while i < len(text)-1:
        if text[i].pos_ == 'NOUN':
            nouns.append(text[i])
        i=i+1
    return nouns